
# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
# !tar xf spark-3.0.1-bin-hadoop2.7.tgz
# !pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [1]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "../spark-3.0.1-bin-hadoop2.7"

Run a local spark session to test your installation:

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Downloaded data

In [1]:
# ! wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2020-08-18/data/listings.csv
# ! wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2020-08-18/data/reviews.csv
# ! wget http://data.insideairbnb.com/spain/catalonia/barcelona/2020-09-12/visualisations/listings.csv

--2020-11-03 20:42:45--  http://data.insideairbnb.com/spain/catalonia/barcelona/2020-09-12/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.77.19
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.77.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3137507 (3,0M) [application/csv]
Saving to: ‘listings.csv’

listings.csv        100%[===================>]   2,99M   836KB/s    in 3,7s    

2020-11-03 20:42:50 (836 KB/s) - ‘listings.csv’ saved [3137507/3137507]



In [29]:
import pandas as pd
import tqdm

In [3]:
listings = spark.read.csv('listings.csv',inferSchema=True, header =True)
# reviews = spark.read.csv('reviews.csv',inferSchema=True, header =True)

train = spark.read.parquet('train_test_data/train.pkt')
test = spark.read.parquet('train_test_data/test.pkt')

In [21]:
# listings_df_pd = pd.read_csv("listings.csv")
# listings_df_pd.to_parquet("listing.pkt")
# listings = spark.read.parquet("listing.pkt")
# listings_not_null = listings[listings.name.isNotNull()]

In [121]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import BucketedRandomProjectionLSH


tokenizer = Tokenizer(inputCol="name", outputCol="CleanTokens")
stopwordsremover = StopWordsRemover(inputCol="CleanTokens", outputCol="CleanTokensStopRemoved")
hashingTF = HashingTF(inputCol="CleanTokensStopRemoved", outputCol="VectorSpace")
idf = IDF(inputCol="VectorSpace", outputCol="VectorSpaceIDF")



In [122]:
pipeline = Pipeline(stages=[tokenizer, stopwordsremover, hashingTF, idf])
pipelineModel = pipeline.fit(train)

In [123]:
df_pipe = pipelineModel.transform(train)

Learning LSH

In [124]:
brp = BucketedRandomProjectionLSH(inputCol="VectorSpaceIDF", outputCol="hashes", bucketLength=2.0,
                                  numHashTables=3)
model = brp.fit(df_pipe)
df_pipe = model.transform(df_pipe)


In [70]:
# test = df_pipe.sample(0.2)

In [180]:
pipelineModelTest = pipeline.fit(test)
test_prepared = pipelineModel.transform(test)

In [181]:
test_pd = test_prepared.toPandas()

In [182]:
looking_for = test_pd.VectorSpaceIDF.to_list()

In [136]:
def find_neighbours(data, value, number, colName):
    result = model.approxNearestNeighbors(data, value, number, distCol=colName)
  
    return result.select("id").toPandas()['id'][1:].to_list()



In [140]:
out = []
for key in tqdm.notebook.tqdm(looking_for):
    neigh = find_neighbours(test_prepared, key, 6, 'hashes')
    out.append(neigh)

<ipython-input-140-117033a312df>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key in tqdm.tqdm_notebook(looking_for):


In [141]:
# predict = [find_neighbours(test_prepared, key, 6, 'hashes') for key in looking_for]

In [164]:
import numpy as np

In [183]:
def compare_lists(a,b):
    results = []
    for i, _ in enumerate(a):
        intersection = set(a[i]).intersection(b[i])
        results.append(len(intersection))
        
    return np.mean(results), results

In [184]:
score, all_res = compare_lists(out, test_pd['ground_truth'])

In [185]:
score

1.714323467230444

In [178]:
3129/3784 * 100

82.69027484143763

In [ ]:
grid_bucket_length = [2, 6, 10]
grid_num_hash_tables = [3, 7, 10]

def find_neighbours(model_, data, value, number, colName):
    result = model_.approxNearestNeighbors(data, value, number, distCol=colName)
  
    return result.select("id").toPandas()['id'][1:].to_list()

def compare_lists(a,b):
    results = []
    for i, _ in enumerate(a):
        intersection = set(a[i]).intersection(b[i])
        results.append(len(intersection))
        
#     return np.mean(results), results
    return np.mean(results), np.sum(np.array(results) > 1)/len(b), np.sum(np.array(results) == 5)/len(b), results


def grid_search_lsh(train_data, test_data, grid_bucket_length, grid_num_hash_tables, targets, limit=100):
    results = []  
    for bucket_length in grid_bucket_length:
        for n_hash_table in grid_num_hash_tables: 
            brp = BucketedRandomProjectionLSH(inputCol="VectorSpaceIDF", 
                                              outputCol="hashes", 
                                              bucketLength=bucket_length,
                                              numHashTables=n_hash_table)
            # fit train
            model = brp.fit(train_data)
            print(f'Models params: \nbucket: {model.getBucketLength()}, n_ht: {model.getNumHashTables()}')
            
            df_pipe = model.transform(train_data)
            looking_for = targets.VectorSpaceIDF.to_list()
            print(f'Calculating LSH for bucket_length={bucket_length} and numHashTables = {n_hash_table}')
            test = test_data.limit(limit)
            targ = looking_for[:limit]
            prediction = []
            
            for key in tqdm.tqdm_notebook(targ):
                neigh = find_neighbours(model, test, key, 6, 'hashes')
                prediction.append(neigh)

            score_a, score_b, score_c, num_neighb = compare_lists(prediction, targets['ground_truth'])
            print(f'Total score: {score_a}, {score_b}, {score_c}\n {num_neighb[:20]}')
            results.append([bucket_length, n_hash_table, score_a, score_b, score_c, num_neighb])
    return results


In [ ]:
res = grid_search_lsh(df_pipe, test_prepared, grid_bucket_length, grid_num_hash_tables, test_pd)